In [33]:
import requests
import pandas as pd
import numpy as np
import time


from IPython.display import clear_output

In [34]:
def getData(ticker, start , end):
    #Create a query list

    past5 = pd.DataFrame(pd.date_range(start, end))
    query_list = []
    temp = []
    count = 0
    for row in past5[0]:
        count += 1
        temp.append(str(row.date()))
        if count == 20:
            count = 0
            query_list.append(temp)
            temp = []

    #Grab data using API for tiingo

    final_data = pd.DataFrame() 
    count = 0
    for query in query_list:
        count += 1
        clear_output()
        print(count)
        start = query[0]
        end = query[-1]
        url = f'https://api.tiingo.com/iex/{ticker}/prices?startDate={start}&endDate={end}&resampleFreq=1min&afterHours=false&columns=open,high,low,close,volume&token=628a6b8ff27cc43ad44f6f69e72f528e30c856b5'
        response = requests.get(url)
        print(response)
        results = pd.DataFrame(response.json())

        #Format the raw data to be readable as open, close, low, high, volume, date, time

        dates = []
        times = []
        for row in range(len(results)):
            datetime = results.iloc[row]['date']
            date = datetime[:10]
            time = datetime[11:23]
            dates.append(date)
            times.append(time)
        results2 = results.drop(columns=['date'])
        results2['date'] = dates
        results2['time'] = times
        results2.columns = ['OPEN', 'HIGH', 'LOW', 'CLOSE', 'VOLUME', 'DATE', 'TIME']
        final_data = pd.concat([final_data, results2], axis=0).reset_index(drop=True)

    # Use our format file to find errors or inconsistencies with raw data

    fd = pd.read_csv('Data_Format.csv')
    fd.set_index('DATETIME', inplace=True)

    final_data['DATETIME'] = final_data['DATE'] + ' ' + final_data['TIME']
    final_data.set_index('DATETIME', inplace=True)

    fd.update(final_data)

    # Some ticker data will be missing, replace these with error so we omit them

    fd['OPEN'] = fd['OPEN'].apply(pd.to_numeric, errors='coerce')
    fd['HIGH'] = fd['HIGH'].apply(pd.to_numeric, errors='coerce')
    fd['LOW'] = fd['LOW'].apply(pd.to_numeric, errors='coerce')
    fd['CLOSE'] = fd['CLOSE'].apply(pd.to_numeric, errors='coerce')

    fd['OPEN'] = fd['OPEN'].interpolate(method='linear')
    fd['HIGH'] = fd['HIGH'].interpolate(method='linear')
    fd['LOW'] = fd['LOW'].interpolate(method='linear')
    fd['CLOSE'] = fd['CLOSE'].interpolate(method='linear')
    fd['VOLUME'].fillna(0.0, inplace=True)
    fd['TICKER'] = ticker

    # Save the formatted raw data as a csv
    fd.to_csv(ticker + '.csv')

    return fd

In [43]:
# Pull the data for an inputted ticker and save it as a csv
input_ticker = input("Enter a ticker: ") # Example: GOOG
# start_date = input("\nStart of date range: ") # Example: 2022-01-01
# end_date = input("\nEnd of date range: ") # Example: 2022-12-31
data = getData(input_ticker, '2022-01-01', '2022-12-31')
data

18
<Response [200]>


,OPEN,HIGH,LOW,CLOSE,VOLUME,DATE,TIME,TICKER
DATETIME,,,,,,,,
2022-01-03 14:30:00.000,NaN,NaN,NaN,NaN,0.0,2022-01-03,14:30:00.000,RUSHB
2022-01-03 14:31:00.000,NaN,NaN,NaN,NaN,0.0,2022-01-03,14:31:00.000,RUSHB
2022-01-03 14:32:00.000,NaN,NaN,NaN,NaN,0.0,2022-01-03,14:32:00.000,RUSHB
2022-01-03 14:33:00.000,NaN,NaN,NaN,NaN,0.0,2022-01-03,14:33:00.000,RUSHB
2022-01-03 14:34:00.000,NaN,NaN,NaN,NaN,0.0,2022-01-03,14:34:00.000,RUSHB
...,...,...,...,...,...,...,...,...
2023-08-11 19:55:00.000,56.3,56.3,56.3,56.3,0.0,2023-08-11,19:55:00.000,RUSHB
2023-08-11 19:56:00.000,56.3,56.3,56.3,56.3,0.0,2023-08-11,19:56:00.000,RUSHB
2023-08-11 19:57:00.000,56.3,56.3,56.3,56.3,0.0,2023-08-11,19:57:00.000,RUSHB
